<a href="https://colab.research.google.com/github/Hamideh-ghanadian/ChatGPT_for_Suicide_Risk_Assessment_on_Social_Media/blob/main/Albert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ALBERT 

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import pandas as pd
from datasets import load_dataset

In [ ]:
data = pd.read_csv('/content/drive/labeled-data.csv', encoding='UTF-8')

In [ ]:
clean=data.dropna()
data= clean.reset_index(drop = True)
data.head(10)

In [ ]:
labels = list(data['label'].unique())
labels

['a', 'b', 'c', 'd']

In [ ]:
numeric_labels =[labels.index(label) for label in labels]
label_map =  dict(zip(labels, numeric_labels))
label_map

In [ ]:
dataset = []
for index,row in data.iterrows():
  item ={'label': label_map[row['label']],
         'Post':row['post_body']}
  dataset.append(item)

In [ ]:
dataset_df = pd.DataFrame(dataset)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(list(dataset_df['Post'].values), list(dataset_df['label'].values), test_size=0.20, random_state=42)

In [ ]:
train_data = pd.DataFrame()
train_data['Post'] = X_train
train_data['label'] = y_train

test_data = pd.DataFrame()
test_data['Post'] = X_test
test_data['label'] = y_test

In [ ]:
from transformers import AutoTokenizer,AlbertConfig, AlbertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=4)  #https://huggingface.co/models

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_encodings = tokenizer(train_data['Post'].values.tolist(), truncation=True, padding=True,return_tensors="pt", max_length=512)
train_labels =train_data['label'].values.tolist()  #placeholders, not real labels

train_dataset = MyDataset(train_encodings, train_labels)

In [ ]:
train_dataset

In [ ]:
test_encodings = tokenizer(test_data['Post'].values.tolist(), truncation=True, padding=True,return_tensors="pt", max_length=512)
test_labels =test_data['label'].values.tolist()  #placeholders, not real labels

test_dataset = MyDataset(test_encodings, test_labels)

In [ ]:
test_dataset

In [ ]:
!pip install sklearn
import numpy as np
from datasets import load_metric
from sklearn.metrics import precision_recall_fscore_support


def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    metrics = ['recall', 'precision', 'f1']

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    acc = metric.compute(predictions=predictions, references=labels)

    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    # Create a dictionary to store the results
    metric_res = {'accuracy': acc['accuracy'], 'precision': precision, 'recall': recall, 'f1': f1}

    return metric_res


In [ ]:
training_args = TrainingArguments(
    output_dir="./out",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy = "epoch", #To calculate metrics per epoch
    logging_strategy="epoch", #Extra: to log training data stats for loss 
)
import transformers
transformers.logging.set_verbosity_info()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(test_dataset)